In [152]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
import time
from sqlalchemy import create_engine
import json
import pymongo
from pymongo import MongoClient
import re

# Extract CSVs into DataFrames

In [153]:
yelp_file = "../Resources/yelp_data.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [154]:
# All columns
yelp_df.columns

Index(['restaurant_name', 'yelp_url', 'restaurant_rating', 'price_cuisine',
       'address1', 'address2', 'restaurant_website'],
      dtype='object')

In [155]:
# restaurant_name check
yelp_df["restaurant_name"].value_counts()
#yelp_df["restaurant_name"].count()

Sandwiches Unlimited Lunch Box    2
L & L Deli & Catering             1
The Black Horse Tavern & Pub      1
Riviera Maya                      1
Big D’s Hot Dogs                  1
                                 ..
Black River Barn                  1
Malay                             1
Tierras y Sabores Restaurant      1
Paisano’s Pizzeria                1
Blossom Asian Bistro              1
Name: restaurant_name, Length: 237, dtype: int64

# Dropping duplicate restaurant_name value

In [156]:
yelp_df1= yelp_df.drop_duplicates(subset=['restaurant_name'])
yelp_df1

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


# Dropping duplicate address1 value

In [157]:
# address1 check
yelp_df["address1"].value_counts()

1250 Sussex Turnpike     2
330 S Salem St           2
276 E Main St            2
459 Main St              2
2 S Main St              1
                        ..
682 Speedwell Ave        1
300 US Hwy 46            1
Dover Train Station      1
278 Diamond Spring Rd    1
12 Elm St                1
Name: address1, Length: 234, dtype: int64

In [158]:
# address2 check
yelp_df["address2"].value_counts()

Morristown, NJ 07960              32
Randolph, NJ 07869                28
Dover, NJ 07801                   25
Denville, NJ 07834                22
Rockaway, NJ 07866                19
Morris Plains, NJ 07950           16
Wharton, NJ 07885                 13
Ledgewood, NJ 07852               11
Succasunna, NJ 07876               9
Chester, NJ 07930                  8
Mendham, NJ 07945                  7
Lake Hopatcong, NJ 07849           6
Flanders, NJ 07836                 5
Netcong, NJ 07857                  4
Stanhope, NJ 07874                 4
Mine Hill, NJ 07803                4
Hopatcong, NJ 07843                3
Kenvil, NJ 07847                   2
Cedar Knolls, NJ 07927             2
Roxbury Township, NJ 07852         2
Mount Olive Township, NJ 07836     2
Mount Arlington, NJ 07856          2
Rockaway, NJ 07801                 1
Landing, NJ 07850                  1
Rockaway Township, NJ 07801        1
Mountain Lakes, NJ 07046           1
Ironia, NJ 07845                   1
J

In [159]:
yelp_df2= yelp_df.drop_duplicates(subset=['address1'])
yelp_df2

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


# Dropping duplicate restaurant_name and also address1 value

In [160]:
yelp_df3= yelp_df1.drop_duplicates(subset=['address1'])
yelp_df3

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",NaN
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",NaN
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",NaN
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [161]:
# average rating
average_rating = yelp_df3["restaurant_rating"].sum()/yelp_df3["restaurant_name"].count()
print("average_rating")
print(average_rating)


average_rating
3.9914163090128754


# Clean DataFrame

# all the Zip Codes of Morris County.

In [162]:
Zip_Codes=pd.read_html('http://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=nj&StName=&StFIPS=&FIPS=34027')
Zip_Codes

[                                                   0
 0  with(new menuname("GWSub")){ style=menuStyle2;...,
     0                                                  1    2  \
 0 NaN  Zip CodeCityCounty07005 Boonton Morris County ...  ...   
 
                                                    3  
 0  Zip CodeCityCounty07961 Convent Station Morris...  ,
     Zip Code                City         County
 0       7005             Boonton  Morris County
 1       7034       Lake Hiawatha  Morris County
 2       7035        Lincoln Park  Morris County
 3       7045           Montville  Morris County
 4       7046      Mountain Lakes  Morris County
 5       7054          Parsippany  Morris County
 6       7058          Pine Brook  Morris County
 7       7082              Towaco  Morris County
 8       7405              Butler  Morris County
 9       7405            Kinnelon  Morris County
 10      7440          Pequannock  Morris County
 11      7444      Pompton Plains  Morris County
 12    

In [163]:
Zip_Codes["Zip Code"]

TypeError: list indices must be integers or slices, not str

In [164]:
# restaurant_rating check
yelp_df3["restaurant_rating"].count()

233

In [165]:
# There are different length for restaurant_name and address1 values.So,we need to fill the empty values.
new_yelp_df=yelp_df3.fillna(0)
new_yelp_df

,restaurant_name,yelp_url,restaurant_rating,price_cuisine,address1,address2,restaurant_website
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869",Find a Table
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869",sbknj.com
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian",322 S Main St,"Wharton, NJ 07885",4seasonswharton.com
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates",64 E Mcfarlan St,"Dover, NJ 07801",quietmanpub.com
4,Verona Restaurant,https://www.yelp.com/biz/verona-restaurant-ran...,4.5,$$Italian,1171 Sussex Tpke,"Randolph, NJ 07869",veronarestaurant.com
...,...,...,...,...,...,...,...
233,Big D’s Hot Dogs,https://www.yelp.com/biz/big-ds-hot-dogs-dover...,5.0,"$Food Trucks, American (Traditional), Hot Dogs",Dover Train Station,"Dover, NJ 07801",0
234,Luigis Pizza,https://www.yelp.com/biz/luigis-pizza-succasun...,4.0,$Pizza,275 State Rt 10 E,"Succasunna, NJ 07876",0
235,Naranja Juice Bar,https://www.yelp.com/biz/naranja-juice-bar-dov...,5.0,"$Juice Bars & Smoothies, Soup, Sandwiches",6B S Warren St,"Dover, NJ 07801",0
236,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Morristown, NJ 07960",cleanjuice.com/locations/morri…


In [166]:
# dropping null value columns to avoid errors 
#new_yelp_df.dropna(inplace = True) 
#new_yelp_df

In [167]:
# price_cuisine check
new_yelp_df["price_cuisine"].value_counts()

$$Italian                                          12
$$Thai                                              6
$Chinese                                            5
$$Mexican                                           5
$$Indian                                            4
                                                   ..
$$Steakhouses, American (Traditional), Barbeque     1
$$American (Traditional), Bars, Seafood             1
$$Bars, Gastropubs, American (New)                  1
$$Chinese, Japanese, Seafood                        1
$$Asian Fusion, Japanese, Sushi Bars                1
Name: price_cuisine, Length: 179, dtype: int64

# restaurant_website check

In [169]:
# restaurant_url check
new_yelp_df["restaurant_website"].value_counts()

0                               49
Find a Table                     3
anthonyandsonsbakery.com         2
zuzannasgourmet.com              1
cinnamonindianrestaurant.com     1
                                ..
rockbottomgrill.com              1
dantesnj.com                     1
hunantaste.com                   1
TheodorasRestaurantPizza.com     1
Www.jaspice.com                  1
Name: restaurant_website, Length: 182, dtype: int64

# Split the price_cuisine column in to price_level and cuisine column

In [ ]:
price_level = []
cuisine = []
for index, row in new_yelp_df.iterrows():
    price_cuisine = re.split(r'(\$+)', row['price_cuisine'])
    if len(price_cuisine) == 1:
        price_level.append("")
        cuisine.append(price_cuisine[0])
    else:
        price_level.append(price_cuisine[1])
        cuisine.append(price_cuisine[2])
new_yelp_df['price_level'] = price_level
new_yelp_df['cuisine'] = cuisine
new_yelp_df

# Split the Split address2 to city, state and zip code

In [ ]:
new_yelp_df[['city','state+zipcode']]=new_yelp_df.address2.str.split(",",expand=True,)
new_yelp_df

In [ ]:
new_yelp_df.drop(columns= "state+zip code")

In [ ]:
new_yelp_df["state+zipcode"].str.split(" ",expand=True,)

In [ ]:
new_yelp_df[['state', 'zip code']] = new_yelp_df["state+zipcode"].str.split(" ",expand=True,)

# Store CSV data into a DataFrame

In [ ]:
csv_file = "../Resources/google_data.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df

In [ ]:
# All columns
google_data_df.columns

In [ ]:
# restaurant name check
google_data_df["restaurant_name"].value_counts()

# Dropping duplicate restaurant_name value

In [ ]:
google_data_df1= google_data_df.drop_duplicates(subset=['restaurant_name'])
google_data_df1

In [ ]:
# restaurant address check
google_data_df["address"].value_counts()

# Dropping duplicate restaurant address value

In [ ]:
google_data_df2= google_data_df.drop_duplicates(subset=['address'])
google_data_df2

# Dropping duplicate restaurant address and restaurant_name value

In [ ]:
google_data_df3= google_data_df1.drop_duplicates(subset=['address'])
google_data_df3

# Clean DataFrame

In [ ]:
# price_level check
google_data_df3["price_level"].value_counts()

In [ ]:
# restaurant_rating check
google_data_df3["restaurant_rating"].value_counts()

In [ ]:
new_google_data_df=google_data_df3.fillna(0)
new_google_data_df

# Create database connection

# Creating a Collection